In [1]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import dlib
import itertools
from joblib import Parallel, delayed  # Import parallel processing

# Load the Dlib facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Paths to datasets
down_syndrome_path = 'downSyndrome'
healthy_path = 'healty'

# Size of patches around landmarks (to be varied)
patch_sizes = [16, 20, 24, 32]

# List of specific landmarks to extract patches from
landmark_indices = [
    36, 39, 42, 45, 27, 30, 33, 31, 35, 51, 48, 54, 57, 68  
]

# Check if the face is frontal by verifying symmetry
def is_frontal_face(landmarks):
    left_eye = np.mean(np.array(landmarks[36:42]), axis=0)
    right_eye = np.mean(np.array(landmarks[42:48]), axis=0)
    nose_tip = np.array(landmarks[30])

    # Calculate distances for symmetry
    eye_distance = np.linalg.norm(left_eye - right_eye)
    nose_to_left_eye = np.linalg.norm(nose_tip - left_eye)
    nose_to_right_eye = np.linalg.norm(nose_tip - right_eye)

    # Symmetry check threshold
    symmetry_threshold = 0.15 * eye_distance
    return abs(nose_to_left_eye - nose_to_right_eye) < symmetry_threshold

# Function to extract patches around specific landmarks
def extract_patches(image, landmarks, indices, patch_size):
    patches = []
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    for idx in indices:
        (x, y) = landmarks[idx]
        # Define the patch region
        x_start = max(x - patch_size // 2, 0)
        y_start = max(y - patch_size // 2, 0)
        x_end = min(x + patch_size // 2, gray.shape[1])
        y_end = min(y + patch_size // 2, gray.shape[0])

        # Extract patch
        patch = gray[y_start:y_end, x_start:x_end]
        if patch.size > 0:
            patches.append(patch)

    return patches

# Function to extract LBP features from patches
def extract_lbp_from_patches(patches, radius, points, method):
    lbp_features = []
    for patch in patches:
        lbp = local_binary_pattern(patch, points, radius, method)
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, points + 3), range=(0, points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)  # Normalize the histogram
        lbp_features.extend(hist)  # Add the LBP histogram to the feature set
    return lbp_features

# Function to get landmarks
def get_landmarks(image_input):
    if isinstance(image_input, str):
        img = cv2.imread(image_input)
    else:
        img = image_input
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(68)]
        
        # Add the extra landmark at the midpoint between 21 and 22
        midpoint_x = (points[21][0] + points[22][0]) // 2
        midpoint_y = (points[21][1] + points[22][1]) // 2
        points.append((midpoint_x, midpoint_y))
        
        # Check if the face is frontal
        if is_frontal_face(points):
            return points
    return None

# Function to crop face from image
def crop_face(image, landmarks, padding_percentage=0.1):
    min_x = min(landmarks, key=lambda x: x[0])[0]
    max_x = max(landmarks, key=lambda x: x[0])[0]
    min_y = min(landmarks, key=lambda x: x[1])[1]
    max_y = max(landmarks, key=lambda x: x[1])[1]

    width = max_x - min_x
    height = max_y - min_y
    max_dimension = max(width, height)

    padding = int(max_dimension * padding_percentage)
    center_x = (min_x + max_x) // 2
    center_y = (min_y + max_y) // 2

    available_left = min_x
    available_right = image.shape[1] - max_x
    available_top = min_y
    available_bottom = image.shape[0] - max_y

    if available_left >= padding and available_right >= padding and available_top >= padding and available_bottom >= padding:
        actual_padding = padding
    else:
        actual_padding = min(available_left, available_right, available_top, available_bottom)

    size = max_dimension + actual_padding * 2

    new_min_x = max(center_x - size // 2, 0)
    new_max_x = min(center_x + size // 2, image.shape[1])
    new_min_y = max(center_y - size // 2, 0)
    new_max_y = min(center_y + size // 2, image.shape[0])

    cropped_image = image[new_min_y:new_max_y, new_min_x:new_max_x]
    cropped_image_resized = cv2.resize(cropped_image, (300, 300))

    return cropped_image_resized

# Function to detect landmarks and extract LBP features from specified points
def get_lbp_features(image, radius, points, method, patch_size):
    # Get landmarks from the original image
    landmarks = get_landmarks(image)
    
    if landmarks:
        # Crop the face from the image based on detected landmarks
        cropped_image = crop_face(image, landmarks)
        
        # Get landmarks again on the cropped image
        cropped_landmarks = get_landmarks(cropped_image)  # Re-apply landmark detection to cropped image
        
        if cropped_landmarks:
            # Extract patches from the cropped image using the new landmarks
            patches = extract_patches(cropped_image, cropped_landmarks, landmark_indices, patch_size)
            
            # Extract LBP features from the patches
            lbp_features = extract_lbp_from_patches(patches, radius, points, method)
            return lbp_features
    
    return None


# Define ranges for RADIUS, POINTS, and PATCH_SIZE
radii = [1, 2, 3, 4]
points_per_radius = [8, 16, 24, 32]
patch_sizes = [16, 20, 24, 32]
method = 'uniform'

# Store results for analysis
results = []

# Function to process each combination of parameters
def process_combination(radius, points, patch_size):
    print(f"Testing RADIUS={radius}, POINTS={points}, PATCH_SIZE={patch_size}...")
    
    # Collect LBP features for current settings
    X = []
    y = []
    for path, label in [(down_syndrome_path, 1), (healthy_path, 0)]:
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)
            img = cv2.imread(img_path)
            if img is not None:
                lbp_features = get_lbp_features(img, radius, points, method, patch_size)
                if lbp_features:
                    X.append(lbp_features)
                    y.append(label)

    # Convert to numpy arrays and scale
    X = np.array(X)
    y = np.array(y)
    X = np.nan_to_num(X)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Train and evaluate the model
    model = SVC(C=1, kernel='rbf', gamma='scale', probability=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate metrics
    accuracy = accuracy_score(y_test, y_pred)
    return (radius, points, patch_size, accuracy)

# Use Parallel from joblib to run the combinations in parallel
results = Parallel(n_jobs=-1)(delayed(process_combination)(radius, points, patch_size)
                               for radius, points, patch_size in itertools.product(radii, points_per_radius, patch_sizes))

# Find the best combination
best_combination = max(results, key=lambda x: x[3])
print("\nBest combination:")
print(f"RADIUS={best_combination[0]}, POINTS={best_combination[1]}, PATCH_SIZE={best_combination[2]}, Accuracy={best_combination[3]:.4f}")


Testing RADIUS=1, POINTS=8, PATCH_SIZE=16...
Testing RADIUS=1, POINTS=8, PATCH_SIZE=20...
Testing RADIUS=1, POINTS=8, PATCH_SIZE=24...
Testing RADIUS=1, POINTS=8, PATCH_SIZE=32...
Testing RADIUS=1, POINTS=16, PATCH_SIZE=16...
Testing RADIUS=1, POINTS=16, PATCH_SIZE=20...
Testing RADIUS=1, POINTS=16, PATCH_SIZE=24...
Testing RADIUS=1, POINTS=16, PATCH_SIZE=32...
Testing RADIUS=1, POINTS=24, PATCH_SIZE=16...
Testing RADIUS=1, POINTS=24, PATCH_SIZE=20...
Testing RADIUS=1, POINTS=24, PATCH_SIZE=24...
Testing RADIUS=1, POINTS=24, PATCH_SIZE=32...
Testing RADIUS=1, POINTS=32, PATCH_SIZE=16...
Testing RADIUS=1, POINTS=32, PATCH_SIZE=20...
Testing RADIUS=1, POINTS=32, PATCH_SIZE=24...
Testing RADIUS=1, POINTS=32, PATCH_SIZE=32...
Testing RADIUS=2, POINTS=8, PATCH_SIZE=16...
Testing RADIUS=2, POINTS=8, PATCH_SIZE=20...
Testing RADIUS=2, POINTS=8, PATCH_SIZE=24...
Testing RADIUS=2, POINTS=8, PATCH_SIZE=32...
Testing RADIUS=2, POINTS=16, PATCH_SIZE=16...
Testing RADIUS=2, POINTS=16, PATCH_SIZE=20

/home/doni/.local/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Testing RADIUS=4, POINTS=8, PATCH_SIZE=16...
Testing RADIUS=4, POINTS=8, PATCH_SIZE=20...
Testing RADIUS=4, POINTS=8, PATCH_SIZE=24...
Testing RADIUS=4, POINTS=8, PATCH_SIZE=32...
Testing RADIUS=4, POINTS=16, PATCH_SIZE=16...
Testing RADIUS=4, POINTS=16, PATCH_SIZE=20...
Testing RADIUS=4, POINTS=16, PATCH_SIZE=24...
Testing RADIUS=4, POINTS=16, PATCH_SIZE=32...
Testing RADIUS=4, POINTS=24, PATCH_SIZE=16...
Testing RADIUS=4, POINTS=24, PATCH_SIZE=20...
Testing RADIUS=4, POINTS=24, PATCH_SIZE=24...
Testing RADIUS=4, POINTS=24, PATCH_SIZE=32...
Testing RADIUS=4, POINTS=32, PATCH_SIZE=16...
Testing RADIUS=4, POINTS=32, PATCH_SIZE=20...
Testing RADIUS=4, POINTS=32, PATCH_SIZE=24...
Testing RADIUS=4, POINTS=32, PATCH_SIZE=32...

Best combination:
RADIUS=1, POINTS=8, PATCH_SIZE=32, Accuracy=0.8246
